# Project Data Type

Objectif : trier des données textuelles par type.  
Les types possibles sont :  
2 : code postal  
3 : coordonnées GPS  
4 : adresse  
5 : numéro SIRET  
6 : Année  
7 : Date  
8 : SIREN  
9 : Code NAF/APE  
10 : Autre  
11 : Téléphone  

In [107]:
# Pour manipuler et analyser des données sous formes de tableaux, on utilise pandas
import pandas as pd
# Pour importer les fichiers CSV, on utilise glob
import glob

In [108]:
# On récupère tous les CSV situés dans le répertoire "./data" du Jupyter notebook
files = glob.glob("./data/*.csv")

# On visualise les colonnes du jeu de données pour voir un peu la tête que ça a pour chaque CSV
for file in files:
    df = pd.read_csv(file, sep=";")
    df.info()
    # Ajout d'un séparateur pour mieux séparer les changements de CSV visuellement
    print("---------")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13194 entries, 0 to 13193
Data columns (total 29 columns):
Dispositif                                                                   13194 non-null object
Type de structure                                                            13194 non-null object
Année d'agrément                                                             13194 non-null object
Localisation                                                                 10167 non-null object
Début d'agrément                                                             13194 non-null int64
Fin d'agrément                                                               13194 non-null int64
Numéro SIREN                                                                 10107 non-null float64
Nom de l'entreprise                                                          13194 non-null object
Sigle                                                                        4021 non-null object
Ac

In [109]:
# Ici, je vérifie manuellement de quel type semble être la donnée (exemple ici pour le CSV 2)
df = pd.read_csv(files[1], sep=";")
df.sample(5)

,ID,Date,Axe,Départ,Arrivée,Nombre de trains programmés,Nombre de trains ayant circulé,Nombre de trains annulés,Nombre de trains en retard à l'arrivée,Taux de régularité,Nombre de trains à l'heure pour un train en retard
2420,INT_16_A,2017-12,Atlantique Méd.,HENDAYE,TOULOUSE MATABIAU,35.0,35.0,0.0,4.0,88.6,7.8
2315,INT_22_A,2017-05,Centre Sud-Ouest,PARIS AUSTERLITZ,TOURS,109.0,109.0,0.0,26.0,76.1,3.2
805,INT_14_A,2014-05,Est Centre,CLERMONT FERRAND,PARIS BERCY,285.0,284.0,1.0,10.0,96.5,27.4
2128,INT_7_B,2018-01,Nord,PARIS-NORD,BOULOGNE-VILLE,152.0,147.0,5.0,21.0,85.7,6.0
1850,INT_7_B,2016-06,Nord-Normandie,PARIS NORD,BOULOGNE,73.0,73.0,0.0,6.0,91.8,11.2


Dans cet exemple, je ne suis intéressé que par la colonne date globalement

In [110]:
# On crée un pseudo-switch qui permet de connaître le type de la donnée en fonction du nom de la colonne du CSV
# Je procède via un grand nombre de "elif" car il n'y a pas de switch/case en Python
# Et que je galère pas mal avec les dictionnaires par manque d'habitude
def returnType(colName):
    # Code postaux
    if colName in ["Code commune (France)"]:
        return 2
    # Coordonnées GPS
    elif colName in ["Géolocalisation", "coord"]:
        return 3
    # Adresses
    elif colName in ["L4_NORMALISEE", "L6_NORMALISEE"]:
        return 4
    # Numéros SIRET
    elif colName in ["SIRETPS"]:
        return 5
    # Années
    elif colName in ["Début d'agrément", "Fin d'agrément"]:
        return 6
    # Dates
    elif colName in ["Date", "JOUR"]:
        return 7
    # Numéros SIREN
    elif colName in ["Numéro SIREN", "SIREN"]:
        return 8
    # Codes NAF/APE
    elif colName in ["Base SIRENE : Code APE de l'établissement siège"]:
        return 9
    # Téléphones
    elif colName in ["TEL", "Téléphone"]:
        return 11
    # Defaut
    else:
        return 0

In [111]:
# On crée une liste temporaire destinée à recevoir les contenus
tempList = []

# Pour chaque CSV, on crée une DataFrame
for file in files:
    print("iterating on file " + file)
    df = pd.read_csv(file, sep=";")
    # Pour chaque colonne du DataFrame actuellement parcouru
    for col in df.columns:
        # On check si la colonne renvoi un type attendu (ou 0 par défaut)
        if returnType(col) != 0:
            print("processing column " + col)
            # Si le type est attendu, on va récupérer tout le contenu de la colonne ainsi que l'indice de type correspondant
            indice = returnType(col)
            valeurs = df[col].tolist()
            # Pour chaque valeur, on va imprimer dans la liste temporaire [indice, valeur]
            for valeur in valeurs:
                tempList.append([indice, str(valeur)])

# Lorsque tous les CSV sont traités, on transforme la liste temporaire en DataFrame
dfsum = pd.DataFrame(tempList, columns=["y", "X"])

iterating on file ./data\fr-esr-cir-et-cii-organismes-et-bureaux-de-style-agrees.csv
processing column Début d'agrément
processing column Fin d'agrément
processing column Numéro SIREN
processing column Téléphone
processing column Géolocalisation
processing column Code commune (France)
processing column Base SIRENE : Code APE de l'établissement siège
iterating on file ./data\regularite-mensuelle-intercites.csv
processing column Date
iterating on file ./data\sirc-17804_9075_61171_2018102_E_Q_20180413_015943916.csv
processing column SIREN
processing column L4_NORMALISEE
processing column L6_NORMALISEE
processing column SIRETPS
processing column TEL
iterating on file ./data\telecom_qualite_services_mobiles_07-2016.csv
processing column coord
iterating on file ./data\validations-sur-le-reseau-de-surface-nombre-de-validations-par-jour-1er-semestr0.csv
processing column JOUR


In [112]:
# On affiche le contenu d'une 15aine de lignes, on doit voir un peu de tout (dates, coordonnées GPS, adresses, etc)
dfsum.sample(15)

,y,X
22954,6,2016
258102,3,"43.20906, 5.6278"
253646,3,"48.80823, 2.99401"
354630,3,"48.085, 3.5147"
179888,3,"42.13431, 9.14056"
287624,3,"50.713695, 2.69201"
332236,3,"43.84776, 4.39081"
285775,3,"48.36411, 1.70077"
192291,3,"48.30729, 1.86196"
273197,3,"50.16453, 1.72202"


A partir de maintenant, je vais utiliser ce dfsum (le DataFrame qui contient la somme traitée de mes CSV) pour entrainer mon machine learning.
Pour des raisons de simplicité je renomme mon dfsum en df.

In [113]:
df = dfsum

## C'est parti pour le machine learning

In [114]:
# On importe une fonction qui convertit des contenus de documents texte en une matrice de comptage de mots/caractères
from sklearn.feature_extraction.text import CountVectorizer

In [115]:
# On détermine qu'un token (fragment de mot) fais la taille d'un mot (à partir de 2 caractères)
vectorizer = CountVectorizer(
    input="content",
    analyzer="char",
    strip_accents=None,
    stop_words=None,
    binary=True,
    ngram_range=(1,2)
)

In [116]:
# On tokenize (on "distingue" les mots/caractères de la donnée)
tokenizer = vectorizer.build_tokenizer()

In [117]:
# On analyse (on distingue des suites de n mots/caractères)
analyzer = vectorizer.build_analyzer()

In [118]:
# On importe une fonction qui segmente les données en jeux d'entrainement et de tests)
from sklearn.model_selection import train_test_split

In [119]:
X = df['X']
y = df['y']

# A l'aide de train_test_split, on va éclater le jeu de données en données d'entrainement (70%) et de test (le reste)
# Pour éviter un avertissement récurrent "FutureWarning", je fixe le jeu de test à 0.3 (le reste)
X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    y,
    train_size = 0.7
)

C:\Users\Xepyon\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [120]:
# On jette un oeil au format des données d'entrainement de X
df.head()

,y,X
0,6,2012
1,6,2012
2,6,2012
3,6,2012
4,6,2012


In [121]:
# On importe une fonction qui applique des transformations aux données afin d'obtenir une évaluation finale
from sklearn.pipeline import Pipeline
# On importe un classifieur pour les modèles Multinomiaux
from sklearn.naive_bayes import MultinomialNB

In [122]:
# On crée un pipeline de traitement, puis on vectorise les données textuelles et enfin on les classifie
classifier = MultinomialNB()
pipeline = Pipeline(
    [
        ("vectorizer", vectorizer),
        ("classifier", classifier)
    ]
)

In [123]:
# On entraine le classifier sur les données d'entrainement
pipeline.fit(X_train.values, Y_train.values)
# Comme ça peut prendre un petit moment, on imprime un "fit done" à la fin pour avertir de l'arrêt du traitement
print("fit done")

fit done


In [124]:
# On utilise le pipeline entrainé sur les données de test
Y_predicted = pipeline.predict(X_test)

In [125]:
# On regarde les résultats des 10 premières lignes
Y_predicted[1:10]

array([ 4,  8,  6,  3,  3,  3,  2,  3, 11], dtype=int64)

In [126]:
# On importe une fonction qui calcule le f1_score (le rapport entre la précision et le rappel)
from sklearn.metrics import f1_score
# On calcule le f1_score pour toutes les prédictions effectuées
score = f1_score(
    Y_test,
    Y_predicted,
    average="micro"
)
print("score f1", score)

score f1 0.9200030633865738


Le score f1 du jeu d'entraînement est de 0.92 soit 92% de réussite

In [127]:
# Maintenant on va tester la même chose sur les données qui sont réellement à évaluer (les données de test)
Y_predicted_train = pipeline.predict(X_train)
# Calcul du score f1
score = f1_score(
    Y_train,
    Y_predicted_train,
    average="micro"
)
print("score f1", score)

score f1 0.921748612357133


Le score f1 du jeu de test est de 0.9217 soit 92.17% de réussite

In [128]:
# Pour s'amuser un peu (et pour voir si les données soumises sont correctement évaluées accessoirement)
# on va permettre de tester directement dans le notebook via un input
dataToTest = input("Insert data to test : ")
# On "cast" les data à tester en Array via [] car predict attend un iterable
dataType = pipeline.predict([dataToTest])
# On crée un dictionnaire de données pour avoir la correspondance indice/type de donnée
typeDict = {
    2 : "code postal",
    3 : "coordonnées GPS",
    4 : "adresse",
    5 : "numéro SIRET",
    6 : "Année",
    7 : "Date",
    8 : "SIREN",
    9 : "Code NAF/APE",
    11 : "Téléphone"
}
print("Data type is : " + typeDict.get(int(dataType)))

Insert data to test : 49100
Data type is : code postal


In [129]:
# On importe la matrice de confusion (vrai positifs/vrai négatifs/faux positifs/faux négatifs)
from sklearn.metrics import confusion_matrix

In [130]:
# La matrice de confusion est très utile car elle permet de voir les vrais/faux positifs/négatifs
print(confusion_matrix(Y_test.values, Y_predicted))

[[ 1507   332     0     0    64     1  1083    50   945]
 [    0 66451     0     0     0     0     0     0     0]
 [    0     0  7585     0     0     0     0     1    35]
 [    0     0     0     0     0     0   334     0  3600]
 [    0     0     0     0  7869    21     0     0     0]
 [    0     0     0     0     0  8218     0     0     0]
 [  456    28     0     0     0     0  6375     0   937]
 [   47     0     0     0     0     0     1  2729  1113]
 [    0     3     0     0     0     0   350     0  7382]]
